In [ ]:
#############################################
# Kural Tabanlı Sınıflandırma ile Potansiyel Müşteri Getirisi Hesaplama
#############################################

#############################################
# İş Problemi
#############################################
# Bir oyun şirketi müşterilerinin bazı özelliklerini kullanarak seviye tabanlı (level based) yeni müşteri tanımları (persona)
# oluşturmak ve bu yeni müşteri tanımlarına göre segmentler oluşturup bu segmentlere göre yeni gelebilecek müşterilerin şirkete
# ortalama ne kadar kazandırabileceğini tahmin etmek istemektedir.

# Örneğin: Türkiye’den IOS kullanıcısı olan 25 yaşındaki bir erkek kullanıcının ortalama ne kadar kazandırabileceği belirlenmek isteniyor.


#############################################
# Veri Seti Hikayesi
#############################################
# Persona.csv veri seti uluslararası bir oyun şirketinin sattığı ürünlerin fiyatlarını ve bu ürünleri satın alan kullanıcıların bazı
# demografik bilgilerini barındırmaktadır. Veri seti her satış işleminde oluşan kayıtlardan meydana gelmektedir. Bunun anlamı tablo
# tekilleştirilmemiştir. Diğer bir ifade ile belirli demografik özelliklere sahip bir kullanıcı birden fazla alışveriş yapmış olabilir.

# Price: Müşterinin harcama tutarı
# Source: Müşterinin bağlandığı cihaz türü
# Sex: Müşterinin cinsiyeti
# Country: Müşterinin ülkesi
# Age: Müşterinin yaşı

################# Uygulama Öncesi #####################

#    PRICE   SOURCE   SEX COUNTRY  AGE
# 0     39  android  male     bra   17
# 1     39  android  male     bra   17
# 2     49  android  male     bra   17
# 3     29  android  male     tur   17
# 4     49  android  male     tur   17

################# Uygulama Sonrası #####################

#       customers_level_based        PRICE SEGMENT
# 0   BRA_ANDROID_FEMALE_0_18  1139.800000       A
# 1  BRA_ANDROID_FEMALE_19_23  1070.600000       A
# 2  BRA_ANDROID_FEMALE_24_30   508.142857       A
# 3  BRA_ANDROID_FEMALE_31_40   233.166667       C
# 4  BRA_ANDROID_FEMALE_41_66   236.666667       C



# Görev 1: Aşağıdaki soruları yanıtlayın

In [50]:
# Soru 1: persona.csv dosyasını okutunuz ve veri seti ile ilgili genel bilgileri gösteriniz.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("persona.csv")

df.info()
df.describe().T, df.head()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


(        count     mean        std   min   25%   50%   75%   max
 PRICE  5000.0  34.1320  12.464897   9.0  29.0  39.0  39.0  59.0
 AGE    5000.0  23.5814   8.995908  15.0  17.0  21.0  27.0  66.0,
    PRICE   SOURCE   SEX COUNTRY  AGE
 0     39  android  male     bra   17
 1     39  android  male     bra   17
 2     49  android  male     bra   17
 3     29  android  male     tur   17
 4     49  android  male     tur   17)

In [9]:
# Soru 2: Kaç unique SOURCE vardır? Frekansları nedir?
print(df['SOURCE'].unique())
print(df['SOURCE'].nunique())
print(df["SOURCE"].value_counts())



['android' 'ios']
2
android    2974
ios        2026
Name: SOURCE, dtype: int64


In [11]:
# Soru 3: Kaç unique PRICE vardır?
print(df['PRICE'].unique())
print(df['PRICE'].nunique())


[39 49 29 19 59  9]
6


In [12]:
# Soru 4: Hangi PRICE'dan kaçar tane satış gerçekleşmiş?
print(df["PRICE"].value_counts())


29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: PRICE, dtype: int64


In [13]:
# Soru 5: Hangi ülkeden kaçar tane satış olmuş?
print(df["COUNTRY"].value_counts())


usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: COUNTRY, dtype: int64


In [14]:
# Soru 6: Ülkelere göre satışlardan toplam ne kadar kazanılmış?
df.groupby(["COUNTRY"]).agg({"PRICE": "sum"})


,PRICE
COUNTRY,
bra,51354
can,7730
deu,15485
fra,10177
tur,15689
usa,70225


In [17]:
# Soru 7: SOURCE türlerine göre göre satış sayıları nedir?
df.groupby(["SOURCE"]).agg({"SOURCE": "count"})


,SOURCE
SOURCE,
android,2974
ios,2026


In [18]:
# Soru 8: Ülkelere göre PRICE ortalamaları nedir?
df.groupby(["COUNTRY"]).agg({"PRICE": "mean"})


,PRICE
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


In [19]:
# Soru 9: SOURCE'lara göre PRICE ortalamaları nedir?
df.groupby(["SOURCE"]).agg({"PRICE": "mean"})


,PRICE
SOURCE,
android,34.174849
ios,34.069102


In [21]:
# Soru 10: COUNTRY-SOURCE kırılımında PRICE ortalamaları nedir?
df.pivot_table(values=['PRICE'],
               index=['COUNTRY'],
               columns=["SOURCE"],
               aggfunc=["mean"])

mean           
             PRICE           
SOURCE     android        ios
COUNTRY                      
bra      34.387029  34.222222
can      33.330709  33.951456
deu      33.869888  34.268817
fra      34.312500  32.776224
tur      36.229437  33.272727
usa      33.760357  34.371703

# GÖREV 2: COUNTRY, SOURCE, SEX, AGE kırılımında ortalama kazançlar nedir?

In [22]:
df.groupby(["COUNTRY","SOURCE","SEX","AGE"]).agg({"PRICE" : "mean"})

PRICE
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897
...                               ...
usa     ios     male   42   30.250000
                       50   39.000000
                       53   34.000000
                       55   29.000000
                       59   46.500000

[348 rows x 1 columns]

# GÖREV 3: Çıktıyı PRICE'a göre sıralayınız.

In [51]:
# Önceki sorudaki çıktıyı daha iyi görebilmek için sort_values metodunu azalan olacak şekilde PRICE'a uygulayınız.
# Çıktıyı agg_df olarak kaydediniz.
agg_df= df.groupby(["COUNTRY","SOURCE","SEX","AGE"]).agg({"PRICE" : "mean"}).sort_values("PRICE", ascending=False)
agg_df

PRICE
COUNTRY SOURCE  SEX    AGE       
bra     android male   46    59.0
usa     android male   36    59.0
fra     android female 24    59.0
usa     ios     male   32    54.0
deu     android female 36    49.0
...                           ...
usa     ios     female 38    19.0
                       30    19.0
can     android female 27    19.0
fra     android male   18    19.0
deu     android male   26     9.0

[348 rows x 1 columns]

# GÖREV 4: Indekste yer alan isimleri değişken ismine çeviriniz.

In [52]:
# Üçüncü sorunun çıktısında yer alan PRICE dışındaki tüm değişkenler index isimleridir.
# Bu isimleri değişken isimlerine çeviriniz.
# İpucu: reset_index()
# agg_df.reset_index(inplace=True)
agg_df.reset_index(inplace=True)
agg_df

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0
...,...,...,...,...,...
343,usa,ios,female,38,19.0
344,usa,ios,female,30,19.0
345,can,android,female,27,19.0
346,fra,android,male,18,19.0


# GÖREV 5: AGE değişkenini kategorik değişkene çeviriniz ve agg_df'e ekleyiniz.

In [53]:
# Age sayısal değişkenini kategorik değişkene çeviriniz.
# Aralıkları ikna edici olacağını düşündüğünüz şekilde oluşturunuz.
# Örneğin: '0_18', '19_23', '24_30', '31_40', '41_70'

agg_df["AGE"].describe()
divs=[0, 18, 23, 30, 40, agg_df["AGE"].max()]
labels=["0_18", "19_23", "24_30", "31_40", "41_"+str(agg_df["AGE"].max())]
agg_df["AGE_CAT"]=pd.cut(agg_df["AGE"], divs, labels=labels)
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT
0,bra,android,male,46,59.0,41_66
1,usa,android,male,36,59.0,31_40
2,fra,android,female,24,59.0,24_30
3,usa,ios,male,32,54.0,31_40
4,deu,android,female,36,49.0,31_40


# GÖREV 6: Yeni level based müşterileri tanımlayınız ve veri setine değişken olarak ekleyiniz.

In [54]:
# customers_level_based adında bir değişken tanımlayınız ve veri setine bu değişkeni ekleyiniz.
# Dikkat!
# list comp ile customers_level_based değerleri oluşturulduktan sonra bu değerlerin tekilleştirilmesi gerekmektedir.
# Örneğin birden fazla şu ifadeden olabilir: USA_ANDROID_MALE_0_18
# Bunları groupby'a alıp price ortalamalarını almak gerekmektedir.
agg_df.columns

Index(['COUNTRY', 'SOURCE', 'SEX', 'AGE', 'PRICE', 'AGE_CAT'], dtype='object')

In [55]:
agg_df = agg_df[['COUNTRY', 'SOURCE', 'SEX', 'AGE_CAT', 'AGE', 'PRICE']]

In [56]:
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE_CAT,AGE,PRICE
0,bra,android,male,41_66,46,59.0
1,usa,android,male,31_40,36,59.0
2,fra,android,female,24_30,24,59.0
3,usa,ios,male,31_40,32,54.0
4,deu,android,female,31_40,36,49.0


In [57]:
agg_df["CUSTOMERS_LEVEL_BASED"] = pd.DataFrame(["_".join(row).upper() for row in agg_df.values[:, 0:4]])

<ipython-input-57-4a44cb4249a6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agg_df["CUSTOMERS_LEVEL_BASED"] = pd.DataFrame(["_".join(row).upper() for row in agg_df.values[:, 0:4]])


In [58]:
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE_CAT,AGE,PRICE,CUSTOMERS_LEVEL_BASED
0,bra,android,male,41_66,46,59.0,BRA_ANDROID_MALE_41_66
1,usa,android,male,31_40,36,59.0,USA_ANDROID_MALE_31_40
2,fra,android,female,24_30,24,59.0,FRA_ANDROID_FEMALE_24_30
3,usa,ios,male,31_40,32,54.0,USA_IOS_MALE_31_40
4,deu,android,female,31_40,36,49.0,DEU_ANDROID_FEMALE_31_40


In [59]:
agg_df = agg_df[["CUSTOMERS_LEVEL_BASED", "PRICE"]].sort_values("PRICE", ascending=False)

In [60]:
agg_df.head()

,CUSTOMERS_LEVEL_BASED,PRICE
0,BRA_ANDROID_MALE_41_66,59.0
2,FRA_ANDROID_FEMALE_24_30,59.0
1,USA_ANDROID_MALE_31_40,59.0
3,USA_IOS_MALE_31_40,54.0
7,USA_ANDROID_FEMALE_41_66,49.0


In [61]:
agg_df = agg_df.groupby("CUSTOMERS_LEVEL_BASED").agg({"PRICE": "mean"}).sort_values("PRICE", ascending=False)
agg_df = agg_df.reset_index()

# GÖREV 7: Yeni müşterileri (USA_ANDROID_MALE_0_18) segmentlere ayırınız.

In [62]:
# PRICE'a göre segmentlere ayırınız,
# segmentleri "SEGMENT" isimlendirmesi ile agg_df'e ekleyiniz,
# segmentleri betimleyiniz, (Segmentlere göre groupby yapıp price mean, max, sum’larınıalınız)
agg_df['SEGMENT'] = pd.qcut(agg_df["PRICE"], 4, labels=["D","C","B","A"])
agg_df.groupby(["SEGMENT"]).agg({"PRICE": ["mean","max","sum"]})

PRICE                        
              mean        max          sum
SEGMENT                                   
D        29.206780  32.333333   817.789833
C        33.509674  34.077340   904.761209
B        34.999645  36.000000   944.990411
A        38.691234  45.428571  1044.663328

# GÖREV 8: Yeni gelen müşterileri sınıflandırınız ne kadar gelir getirebileceğini tahmin ediniz.

In [64]:
# 33 yaşında ANDROID kullanan bir Türk kadını hangi segmente aittir ve ortalama ne kadar gelir kazandırması beklenir?
new_user="TUR_ANDROID_FEMALE_31_40"
agg_df[agg_df["CUSTOMERS_LEVEL_BASED"] == new_user]


,CUSTOMERS_LEVEL_BASED,PRICE,SEGMENT
3,TUR_ANDROID_FEMALE_31_40,41.833333,A


In [65]:
# 35 yaşında IOS kullanan bir Fransız kadını hangi segmente ve ortalama ne kadar gelir kazandırması beklenir?
new_user="FRA_IOS_FEMALE_31_40"
agg_df[agg_df["CUSTOMERS_LEVEL_BASED"] == new_user]

,CUSTOMERS_LEVEL_BASED,PRICE,SEGMENT
78,FRA_IOS_FEMALE_31_40,32.818182,C
